In [1]:
###

##### The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on **the location** and proximities of a **launch site**, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations

In [2]:
# import librairy

import folium
import pandas as pd

from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

import requests
import io

In [3]:
# import Data
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'

response = requests.get(URL)
if response.status_code == 200:
    SpaceX_data = pd.read_csv(io.StringIO(response.text))
else:
    print(f"HTTP GET request failed with status code: {response.status_code}")

In [4]:
SpaceX_data.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [5]:
SpaceX_data = SpaceX_data[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_data = SpaceX_data.groupby(['Launch Site'], as_index=False).first()
launch_sites_data = launch_sites_data[['Launch Site', 'Lat', 'Long']]
launch_sites_data

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [6]:
for index, row in launch_sites_data.iterrows():
    launch_site = row['Launch Site']
    coordinates = (row['Lat'], row['Long'])
    print(f"Coordonnées de {launch_site} sont {coordinates}")

Coordonnées de CCAFS LC-40 sont (28.56230197, -80.57735648)
Coordonnées de CCAFS SLC-40 sont (28.56319718, -80.57682003)
Coordonnées de KSC LC-39A sont (28.57325457, -80.64689529)
Coordonnées de VAFB SLC-4E sont (34.63283416, -120.6107455)


In [7]:

nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [13]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name

circle = folium.Circle(nasa_coordinate, radius=1000, color='#FF0000', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#FF0000;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [9]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name


for index, row in launch_sites_data.iterrows():
    launch_site = row['Launch Site']
    coordinates = [row['Lat'], row['Long']]
    circle = folium.Circle(coordinates, radius=10000, color='#FF0000', fill=True).add_child(folium.Popup(launch_site))
    marker = folium.map.Marker(coordinates, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % launch_site,
                                                       )
    )

    site_map.add_child(circle)
    site_map.add_child(marker)
site_map

In [10]:
marker_cluster = MarkerCluster()

In [11]:
SpaceX_data['marker_color'] = SpaceX_data['class'].apply(lambda x: 'green' if x == 1 else 'red')

In [14]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

for index, record in SpaceX_data.iterrows():
    # Create a Marker object with its coordinate
    coordinates = [record['Lat'], record['Long']]

    # Customize the Marker's icon property based on the 'marker_color' column
    icon_color = record['marker_color']
    marker = folium.Marker(location=coordinates, icon=folium.Icon(color='white', icon_color=icon_color))

    # Add the marker to the MarkerCluster
    marker.add_to(marker_cluster)

site_map